In [40]:
import requests
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
import json
from config import weather_api_key

In [41]:
url = "https://api.openweathermap.org/data/3.0/onecall/timemachine?"
units = "imperial"

In [42]:
# Reading the CSV
uber_csv = pd.read_csv("Resources/uber.csv")

uber_df = pd.DataFrame(uber_csv)

uber_df.head()



,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [43]:
# Renaming the headers
uber_df.rename(columns = {'Unnamed: 0':'ID', 'key':'N/A', 'fare_amount':'Charge', "pickup_datetime":"Pickup Time",  "pickup_longitude": "Pickup Lon", "pickup_latitude":"Pickup Lat",  
                          "dropoff_longitude": "Dropoff Lon", "dropoff_latitude": "Dropoff Lat", "passenger_count": "Passenger Count"}, inplace = True)

uber_df.head()



,ID,N/A,Charge,Pickup Time,Pickup Lon,Pickup Lat,Dropoff Lon,Dropoff Lat,Passenger Count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [44]:
# Grabbing a sample of the data
uber_df3 = uber_df.sample(n=5)
uber_df3

,ID,N/A,Charge,Pickup Time,Pickup Lon,Pickup Lat,Dropoff Lon,Dropoff Lat,Passenger Count
130880,16667675,2010-09-06 00:12:00.00000074,6.5,2010-09-06 00:12:00 UTC,-73.991355,40.732933,-74.010618,40.722430,1
71121,18982346,2014-07-05 19:19:26.0000003,4.5,2014-07-05 19:19:26 UTC,-73.997198,40.742393,-74.001483,40.732196,1
155175,49765371,2010-09-19 08:32:23.0000002,12.1,2010-09-19 08:32:23 UTC,-73.959317,40.767838,-73.985462,40.735685,1
79700,17855548,2014-11-04 19:58:00.00000055,6.5,2014-11-04 19:58:00 UTC,-73.980087,40.765727,-73.990133,40.760678,1
143048,5017353,2010-11-13 21:14:00.000000125,7.3,2010-11-13 21:14:00 UTC,-73.982423,40.771645,-73.970633,40.795072,1


In [45]:
# Formatting the date step one
uber_df3['Date Formatted'] = (pd.to_datetime(uber_df3['Pickup Time'], yearfirst=True, utc=True)) 
 
uber_df3


,ID,N/A,Charge,Pickup Time,Pickup Lon,Pickup Lat,Dropoff Lon,Dropoff Lat,Passenger Count,Date Formatted
130880,16667675,2010-09-06 00:12:00.00000074,6.5,2010-09-06 00:12:00 UTC,-73.991355,40.732933,-74.010618,40.722430,1,2010-09-06 00:12:00+00:00
71121,18982346,2014-07-05 19:19:26.0000003,4.5,2014-07-05 19:19:26 UTC,-73.997198,40.742393,-74.001483,40.732196,1,2014-07-05 19:19:26+00:00
155175,49765371,2010-09-19 08:32:23.0000002,12.1,2010-09-19 08:32:23 UTC,-73.959317,40.767838,-73.985462,40.735685,1,2010-09-19 08:32:23+00:00
79700,17855548,2014-11-04 19:58:00.00000055,6.5,2014-11-04 19:58:00 UTC,-73.980087,40.765727,-73.990133,40.760678,1,2014-11-04 19:58:00+00:00
143048,5017353,2010-11-13 21:14:00.000000125,7.3,2010-11-13 21:14:00 UTC,-73.982423,40.771645,-73.970633,40.795072,1,2010-11-13 21:14:00+00:00


In [46]:
# Final Formatted date added to dataframe
uber_df3['Date Formatted Final'] = (uber_df3["Date Formatted"] - pd.Timestamp("1970-01-01", tz="UTC")) // pd.Timedelta('1s')
uber_df3



,ID,N/A,Charge,Pickup Time,Pickup Lon,Pickup Lat,Dropoff Lon,Dropoff Lat,Passenger Count,Date Formatted,Date Formatted Final
130880,16667675,2010-09-06 00:12:00.00000074,6.5,2010-09-06 00:12:00 UTC,-73.991355,40.732933,-74.010618,40.722430,1,2010-09-06 00:12:00+00:00,1283731920
71121,18982346,2014-07-05 19:19:26.0000003,4.5,2014-07-05 19:19:26 UTC,-73.997198,40.742393,-74.001483,40.732196,1,2014-07-05 19:19:26+00:00,1404587966
155175,49765371,2010-09-19 08:32:23.0000002,12.1,2010-09-19 08:32:23 UTC,-73.959317,40.767838,-73.985462,40.735685,1,2010-09-19 08:32:23+00:00,1284885143
79700,17855548,2014-11-04 19:58:00.00000055,6.5,2014-11-04 19:58:00 UTC,-73.980087,40.765727,-73.990133,40.760678,1,2014-11-04 19:58:00+00:00,1415131080
143048,5017353,2010-11-13 21:14:00.000000125,7.3,2010-11-13 21:14:00 UTC,-73.982423,40.771645,-73.970633,40.795072,1,2010-11-13 21:14:00+00:00,1289682840


In [47]:
#Creating the clean dataframe
clean_df = uber_df3[["ID", "Charge", "Pickup Lon", "Pickup Lat", "Dropoff Lon", "Dropoff Lat", "Passenger Count", "Date Formatted Final" ]]


clean_df

,ID,Charge,Pickup Lon,Pickup Lat,Dropoff Lon,Dropoff Lat,Passenger Count,Date Formatted Final
130880,16667675,6.5,-73.991355,40.732933,-74.010618,40.722430,1,1283731920
71121,18982346,4.5,-73.997198,40.742393,-74.001483,40.732196,1,1404587966
155175,49765371,12.1,-73.959317,40.767838,-73.985462,40.735685,1,1284885143
79700,17855548,6.5,-73.980087,40.765727,-73.990133,40.760678,1,1415131080
143048,5017353,7.3,-73.982423,40.771645,-73.970633,40.795072,1,1289682840


In [48]:
# Making the list for data 
uber_data = []

# Looping through the data
for i, row in clean_df.iterrows():

    # Gathering the latitude, longitude, and pickup time
    lat = row['Pickup Lat']
    lon = row['Pickup Lon']
    date = row['Date Formatted Final'].astype(int)
    
    # Making the query for the search using the varibles
    query_url = f"{url}lat={lat}&lon={lon}&dt={date}&appid={weather_api_key}"

    print(query_url)
    
    # Getting the info from the api
    response = requests.get(query_url).json()

    # Gathering variables for statistics
    temp = response['data'][0]['temp']
    hum = response['data'][0]['humidity']
    desc = response['data'][0]['weather'][0]['description']

    # Placing variables into a dictionary
    uber_data.append({"Pickup Lat": lat,
                            "Temp": temp,
                            "Humidity": hum,
                            "Description": desc
    })


    
    
   

https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=40.732933&lon=-73.991355&dt=1283731920&appid=602989f1437335ec5cd0b77732b10a8a
https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=40.742393&lon=-73.997198&dt=1404587966&appid=602989f1437335ec5cd0b77732b10a8a
https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=40.767838&lon=-73.959317&dt=1284885143&appid=602989f1437335ec5cd0b77732b10a8a
https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=40.76572700000001&lon=-73.980087&dt=1415131080&appid=602989f1437335ec5cd0b77732b10a8a
https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=40.771645&lon=-73.982423&dt=1289682840&appid=602989f1437335ec5cd0b77732b10a8a


In [49]:
# Making the weather dataframe
weather_df = pd.DataFrame(uber_data)

weather_df.head()

,Pickup Lat,Temp,Humidity,Description
0,40.732933,293.09,66,clear sky
1,40.742393,299.42,33,few clouds
2,40.767838,290.24,81,clear sky
3,40.765727,293.30,22,clear sky
4,40.771645,285.91,55,clear sky


In [50]:
# Merging the data for statistical use
final_df = clean_df.merge(weather_df, on = "Pickup Lat")

final_df

,ID,Charge,Pickup Lon,Pickup Lat,Dropoff Lon,Dropoff Lat,Passenger Count,Date Formatted Final,Temp,Humidity,Description
0,16667675,6.5,-73.991355,40.732933,-74.010618,40.722430,1,1283731920,293.09,66,clear sky
1,18982346,4.5,-73.997198,40.742393,-74.001483,40.732196,1,1404587966,299.42,33,few clouds
2,49765371,12.1,-73.959317,40.767838,-73.985462,40.735685,1,1284885143,290.24,81,clear sky
3,17855548,6.5,-73.980087,40.765727,-73.990133,40.760678,1,1415131080,293.30,22,clear sky
4,5017353,7.3,-73.982423,40.771645,-73.970633,40.795072,1,1289682840,285.91,55,clear sky
